# Постановка задачи

![](https://newsland.com/static/u/comment_image_from_text/01042017/87421366-1774244.jpg)    
     
В энергосбытовой компании (далее - ЭСК) есть несознательные абоненты, которые имеют задолженность за потребленную электроэнергию. С этими должниками проводится некоторая работа. А, может, не проводится, потому что часть должников вспоминает о долге и начинает платить по частям, небольшими суммами. С такими должниками обходятся довольно мягко - к ним претензии не предъявляются, от них ждут постепеннной оплаты. Мягкость обхождения с ними имеет экономические причины - работа с должниками обходится ЭСК недешево. Информирование по телефону или по электронной почте, а также взыскание через суд - эти действия при массовых объемах (несколько сотен тысяч должников) тратят значительные ресурсы компании, и не всегда траты оправдывают себя.    
   
Жесткого алгоритма разбиения должников на группы для разных сценариев работы по истребованию долга у ЭСК нет. В основном опираются на сумму задолженности, но она не является главным фактором, потому что, например, 5000 тыс. руб. долга для коттеджа - это долг за 1 месяц, а та же сумма для пенсионера в однокомнатной квартире - это годовая задолженность. Также нельзя опираться и на длительность долга. Некоторые абоненты могли находиться в стесненных обстоятельствах, накопить долги, но после исправления ситуации начать понемногу оплачивать. С такими должниками (а их достаточно много) для снижения социальной напряженности работы по взысканию обычно не проводятся. Их обычно просто информируют, и на этом работа заканчивается. Но есть должники, которые упорно не желают оплачивать, - с такими надо проводить весь спектр работ.    

Необходимо разбить всю массу должников на группы, которые показывали бы их отношение к задолженности и их социальное положение. Данное разбиение должно учитывать факторы, которые может предоставить биллинговая система учета электроэнергии. Результатом должны являться списки должников 3 или 4 видов: должники, не требующие внимания, требующие мягкого/умеренного внимания, требующие пристального внимания. 

![](https://michurinec.org/uploads/images/dolzhnik.jpg)

# Библиотеки и функции

In [ ]:
import numpy as np 
import pandas as pd 
import random
import scipy
from scipy import spatial

import sys
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import homogeneity_score, silhouette_score, completeness_score, v_measure_score

from sklearn import cluster, datasets, mixture
# from sklearn.tree import DecisionTreeClassifier, export_graphviz
# from sklearn.neighbors import kneighbors_graph
# from itertools import cycle, islice
from scipy.spatial.distance import cosine

import warnings
warnings.filterwarnings(action="ignore")

In [ ]:
!pip install SimpSOM
import SimpSOM as sps

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, LeakyReLU, Add, Activation, ZeroPadding2D, Dropout
from keras.layers import BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model, Sequential, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras.initializers import glorot_uniform
from keras.optimizers import SGD

In [ ]:
# Зафиксируем воспроизводимость экспериментов
RANDOM_SEED = 21
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [ ]:
# Функция для сравнения графиков чистых данных и их логарифмов 
def val_log_plot(df, col):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].hist(df[col], rwidth=0.9, alpha=0.7, bins=15)
    ax[0].set_title(col)
    ax[1].hist(np.log(df[col]+1), rwidth=0.9, alpha=0.7, bins=15)
    ax[1].set_title('log of '+col)
    plt.show()
    
    
# Определение выбросов
def get_outlier(df, col):
    Q3 = pd.DataFrame.quantile(df, q=0.75, axis=0, numeric_only=True, interpolation='midpoint')[col]
    Q1 = pd.DataFrame.quantile(df, q=0.25, axis=0, numeric_only=True, interpolation='midpoint')[col]
    IQR = round(Q3-Q1,1)
    return df[~df[col].between(Q1 - 1.5*IQR, Q3 + 1.5*IQR)][col], Q1 - 1.5*IQR, Q3 + 1.5*IQR


# Информация о выбросах с графиками
def show_info(df, col, show=True):
    # Выводим количество выбросов и их границы
    out, lim1, lim2 = get_outlier(df, col)
    minCol = df[col].min()
    maxCol = df[col].max()
    median = df[col].median()
    nulCol = sum(pd.isnull(df[col]))
    
    cnt = min(int(df[col].value_counts().count()),2000)
    
    if show:
        print('Не заполнено: ', nulCol)
        print('Минимум: ', minCol)
        print('Максимум: ', maxCol)
        print('Медиана: ', median)
        print('Количество выбросов: ', len(out))
        if len(out) > 0:
            print('Нижняя граница выбросов: ', lim1)
            print('Верхняя граница выбросов: ', lim2)

        # Выводим графики: гистограмму и боксплот
        fig, axes = plt.subplots(1,2,figsize=(12,4))
        axes[0].hist(df[col], bins=cnt)
        axes[1].boxplot(df[col])
    
    return {'med': median, 'lm1': lim1, 'lm2': lim2}


# Функция генерации произвольного цвета
def generate_color():
    color = '#{:02x}{:02x}{:02x}'.format(*map(lambda x: random.randint(0, 255), range(3)))
    return color


# Функция визуализации кластеров и расчета центроидов
def plot_clusters(df_clust, labels, need_pca=True, name_alorithm = ''):
    
    # Для визуализации кластеров многомерных объектов понизим размерность методом выделения главных компонент
    if need_pca:
        pca = PCA(2)
        pca.fit(df_clust)
        X_PCA = pca.transform(df_clust)
        x, y = X_PCA[:, 0], X_PCA[:, 1]
    else:
        x, y = df_clust[:, 0], df_clust[:, 1]

    # Каждому кластеру назначим свой цвет на графике
    clust = np.unique(labels)
    colors = {}
    if len(clust) == 3:
        colors[clust[0]] = 'red'
        colors[clust[1]] = 'blue'
        colors[clust[2]] = 'green'
    else:
        for i in range(len(clust)):
            colors[clust[i]] = generate_color()

    # Прорисовываем график
    df1 = pd.DataFrame({'x': x, 'y':y, 'label':labels}) 
    groups = df1.groupby('label')
    centroids = {}

    fig, ax = plt.subplots(figsize=(10, 10)) 

    for name, group in groups:
        ax.plot(group.x, group.y, marker='o', linestyle='', ms=4,
                color=colors[name],label='cluster ' + str(name), mec='none', zorder=-1)
        ax.set_aspect('auto')
        ax.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off')
        ax.tick_params(axis= 'y',which='both',left='off',top='off',labelleft='off')
        
        centroid = (sum(group.x)/len(group.x),sum(group.y)/len(group.y))
        centroids[name] = centroid
        ax.scatter(centroid[0], centroid[1], color='black', marker = 'X', s=200, zorder=1)
        
#     for i in range(len(centroids)):
#         ax.scatter(centroids[i][0], centroids[i][1], color='black', zorder=1)
    
    ax.legend()
    ax.set_title(name_alorithm + " Кластеры должников")
    plt.show()
    
    # Проверим метрики кластеризации
    silhouette = silhouette_score(df_clust, labels, metric='euclidean')
    homogeneity = homogeneity_score(labels_true=y, labels_pred=labels)
    completeness = completeness_score(labels_true=y, labels_pred=labels)
    v_measure = v_measure_score(labels_true=y, labels_pred=labels)

    print('silhouette = ', silhouette)
    print('homogeneity = ', homogeneity)
    print('completeness = ', completeness)
    print('v_measure = ', v_measure)
    
    dfc = pd.DataFrame(centroids)
    arr = []
    for i in range(len(dfc.columns)):
        arr.append(dfc[dfc.columns[i]].to_list())
    
    return arr
    
def get_centroid(x,y):
    return sum(x)/len(x),sum(y)/len(y)

# Чтение данных

In [ ]:
# Проверяем, сколько у нас файлов с сырыми данными и какого они формата
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Данные по должникам разбиты по районам Санкт-Петербурга. Районы и их должники подобраны примерно одинаковые, то есть заранее исключены далекие районы в депрессивных пригородах. Остальные районы по опыту работы в ЭСК не слишком отличаются в плане работы с задолженностью, поэтому выделять особый признак для района должника нет надобности.

In [ ]:
# Собираем данные из папки с должниками
data = pd.concat([pd.read_csv(f) for f in glob.glob('../input/debtors/z*.csv')])
data.shape

Датасет собран небольшой. В реальности должников гораздо больше. Данные собирались специально так, чтобы совпадало распределение числовых величин на полном наборе и на выборочном. 

In [ ]:
# Обзор данных
display(data.info())

**Описание полей:**   
    
ACODE_CODE_ORG_UNIT - идентификатор абонента ЭСК     
BU_COUNT - количество актов нарушений учета электроэнергии    
COURT_COUNT - количество судебных дел по взысканию задолженности    
DATE_CONTROL - дата последнего контрольного посещения абонента для проверки счетчика и его показаний     
METER_OK - признак наличия исправного счетчика       
IS_IKUS - признак наличия у абонента личного кабинета на портале ЭСК         
PHONE_OK - признак наличия известного и действительного (не устаревшего) номера домашнего телефона     
MOBILE_OK - признак наличия известного и действительного (не устаревшего) номера мобильного телефона; данный признак заполняется либо по данным из личного кабинета, либо при оплате на сайте ЭСК с указанием номера мобильного телефона (номер можно указать при оплате для получения на него электронной квитанции) - примерно в равном соотношении       
EMAIL_OK - признак наличия известного и действительного (не устаревшего) адреса эл. почты для получения счетов; данный признак заполняется либо по данным из личного кабинета, либо (очень редко) по обращению в ЭСК присылать счета на электронную почту                
SUM_OVERDUE - сумма долга в рублях (далее все суммы даны в рублях)   
DEB_TIME - количество месяцев задолженности     
SHIP_SUM - общая сумма начислений у абонента    
REAL_SUM - общая сумма оплаты у абонента     
PAY_3 - сумма платежей за последние 3 месяца    
PAY_6 - сумма платежей за последние 6 месяцев    
PAY_9 - сумма платежей за последние 9 месяцев    
PAY_12 - сумма платежей за последние 12 месяцев    
PAY_15 - сумма платежей за последние 15 месяцев    
PAY_18 - сумма платежей за последние 18 месяцев    
SHP_3 - сумма начислений за последние 3 месяца    
SHP_6 - сумма начислений за последние 6 месяцев    
SHP_9 - сумма начислений за последние 9 месяцев    
SHP_12 - сумма начислений за последние 12 месяцев    
SHP_15 - сумма начислений за последние 15 месяцев    
SHP_18 - сумма начислений за последние 18 месяцев    
     
**Пояснение по полям:**     
1) Запрос по оплатам и начислениям из рабочей базы данных составлен для последних 18 месяцев. Опыт показывает, что абоненты обычно сами помнят о долгах и выстраивают план их оплаты только в пределах 1,5 лет. Все долги, более ранние, нежели этот срок, взыскать в судебном и досудебном порядке обычно крайне сложно (хотя срок давности = 3 года). Поэтому борьбу с задолженностью ведут обычно в пределах долга за последние 1,5 года.   
    
2) Процесс взыскания задолженности строится в зависимости от суммы и длительности долга абонента, а также от возможности информирования его по разным каналам: письмо по электронной почте, звонок по домашнему телефону, СМС на мобильный телефон, уведомление в личном кабинете, доставка письменного уведомления лично в руки. Данные о возможностях применения тех или иных каналов также включены в датасет (наличие эл. почты, мобильного телефона, регистрации в личном кабинете).  
    
3) В набор данных включена дополнительно информация о состоянии счетчика электроэнергии. Опыт показывает, что те абоненты, у которых счетчик исправен, не безнадежны. Те же, у кого он отсутствует или испорчен, являются злостными и принципиальными неплательщиками. Дополнительно включена информация о наличии актов нарушения учета - самовольного подключения или вмешательства в механизм счетчика для искажения показаний.    
     
4) Посещение контролером абонента на дому и снятие контрольных показаний часто понуждает должника начать оплачивать долг. Поэтому в датасет включена информация о дате последнего контрольного посещения.     
      
5) В датасете присутствует столбец с количеством судебных дел в отношении неплательщика. Обычно взыскание по суду идет в упрощенном порядке: на должника подают заявление о выдаче судебного приказа мировым судьей, сам неплательщик при этом в суд не вызывается, его просто ставят перед фактом судебного дела. Взыскание по суду обычно идет не по желанию должника - через списание из зарплаты/пенсии или с банковского счета. С теми должниками, на кого уже подано в суд, вести какие-то работы уже не имеет смысла. С них и так снимут деньги через службу судебных приставов.

In [ ]:
# Читаем файл с шаблоном действий для каждого варианта разбиения
pattern = pd.read_csv('../input/debt-pattern/Pattern.csv')
pattern

**Описание полей:**    
     
pir - признак кластера с самым большим количеством судебных дел     
big_overdue - признак кластера с самым большим долгом     
not_little_pay - признак кластера не с самой маленькой оплатой (то есть, средней или большой)     
geek - признак кластера не с самым маленьким количеством номеров мобильных телефонов и адресов электронной почты (то есть, со средним или большим)          
need_message - нужно ли при таких характеристиках кластера оповещение о долге (нет/по одному каналу оповещения/по всем каналам оповещения)   
need_pir - нужно ли при таких характеристиках кластера обращение в суд (нет/да)

После выделения кластеров и получения их характеристик в соответствии с данным шаблоном будут даны списки абонентов с рекомендацией по оповещению и обращению в суд.

# EDA & Feature Engineering

### Преобразование строк в числа

Числовые показатели, отвечающие за динамику начислений и оплат, даются в строковом виде. Преобразуем их

In [ ]:
# Числовые столбцы, переданные в строковом виде
num_columns = ['SUM_OVERDUE','SHIP_SUM', 'REAL_SUM', 'PAY_3', 'PAY_6',
       'PAY_9', 'PAY_12', 'PAY_15', 'PAY_18', 'SHP_3', 'SHP_6', 'SHP_9',
       'SHP_12', 'SHP_15', 'SHP_18']

In [ ]:
# Преобразуем строку в число
for col in num_columns:
    data[col] = data[col].apply(lambda x: float(x.replace(',','.')))

In [ ]:
# Смотрим на распределение числовых данных
display(data.describe())

### Признак DATE_CONTROL

Признак с датой последнего контрольного посещения - строковый. Преобразуем его в числовое значение.

In [ ]:
# Смотрим, как выглядит признак с датой
display(data.DATE_CONTROL)
display(data[data.DATE_CONTROL.str.len() > 10].DATE_CONTROL)

Видно, что часть дат идет с временем. Время можно отсечь, так как период до последнего посещения будем измерять в днях.

In [ ]:
# Обрубим время в строке с датой и строку превратим в дату
data['DATE_CONTROL'] = data['DATE_CONTROL'].apply(lambda x: x[:10])
data['DATE_CONTROL'] = data['DATE_CONTROL'].apply(lambda x: datetime.strptime(x, "%d.%m.%Y"))

In [ ]:
# Создадим новый признак - количество дней от даты последнего посещения контролера до даты формирования датасета
max_date = data['DATE_CONTROL'].max()  
data['control_days'] = data['DATE_CONTROL'].apply(lambda x: (max_date-x).days)

### Признак SUM_OVERDUE

In [ ]:
data.SUM_OVERDUE.hist()

In [ ]:
d = show_info(data, 'SUM_OVERDUE')

Видно, что в должники попали абоненты, у которых маленькие суммы долга и даже нулевые суммы. А также есть должники, у которых суммы очень большие. Отсечем тех, у кого долг маленький (порог долга в ЭСК = 1000 руб.), и тех, у кого долг огромный (свыше 100000 руб.; должников с такой суммой в ЭСК знают поименно и работу с ними ведут отдельно, персонально с каждым).

In [ ]:
# Проверим количество должников за пределами порогов
display(data[data.SUM_OVERDUE > 100000].shape[0])
display(data[data.SUM_OVERDUE < 1000].shape[0])

In [ ]:
# Сохраним необрезанные данные и отсечем лишних должников
df = data.copy()
df = df[df.SUM_OVERDUE.between(1000, 100000)]
df.shape[0]

### Признаки SHIP_SUM и REAL_SUM

In [ ]:
df.REAL_SUM.hist()

In [ ]:
df.SHIP_SUM.hist()

Видно, что распределение у данных признаков крайне неравномерное. Но для нас эти признаки совершенно не показательны. поскольку интерес представляет не абсолютная сумма, а относительная - каков процент долга или оплат от суммы всех начислений.

In [ ]:
# Создадим новые признаки для отношения долга и оплат к полной сумме начислений
df['overdue'] = df['SUM_OVERDUE']/df['SHIP_SUM']
df['pay'] = df['REAL_SUM']/df['SHIP_SUM']

In [ ]:
df['pay'].hist(bins=100)

In [ ]:
df['overdue'].hist(bins=100)

По графикам видно, что значительный процент должников имеют задолженность не более 10% от всех сумм начислений.    
Также видно, что две рассмотренные переменные тесно коррелируют между собой. Их расчет достаточно сложный, долг не равен начисления минус оплаты (в долге учитываются дополнительно иные факторы), но для моделирования один из столбцов можно будет удалить.

### Признаки PAY_... и SHP_...

Данные признаки показывают динамику оплат абонентом за последние 1,5 года.    
Их также переведем в относительные величины.

In [ ]:
# Получаем уровень оплат в динамике
df['pay3'] = df['PAY_3']/df['SHP_3']
df['pay6'] = df['PAY_6']/df['SHP_6']
df['pay9'] = df['PAY_9']/df['SHP_9']
df['pay12'] = df['PAY_12']/df['SHP_12']
df['pay15'] = df['PAY_15']/df['SHP_15']
df['pay18'] = df['PAY_18']/df['SHP_18']

In [ ]:
# Проверим, сколько незаполненных данных
print('pay3 is null: ' , len(df[df['pay3'].isnull()]))
print('pay6 is null: ' , len(df[df['pay6'].isnull()]))
print('pay9 is null: ' , len(df[df['pay9'].isnull()]))
print('pay12 is null: ' , len(df[df['pay12'].isnull()]))
print('pay15 is null: ' , len(df[df['pay15'].isnull()]))
print('pay18 is null: ' , len(df[df['pay18'].isnull()]))

Все числовые столбцы с начислениями и оплатами в изначальном датасете были заполнены. Пустые значения в сгенерированных столбцах могли появиться из-за того, что было деление ноль на ноль, то есть, сумма начислений была нулевой. Проверим это.

In [ ]:
# Выбираем все значения SHP_... при pay...=null
print(df[df['pay3'].isnull()].SHP_3.unique())
print(df[df['pay6'].isnull()].SHP_6.unique())
print(df[df['pay9'].isnull()].SHP_9.unique())
print(df[df['pay12'].isnull()].SHP_12.unique())
print(df[df['pay15'].isnull()].SHP_15.unique())
print(df[df['pay18'].isnull()].SHP_18.unique())

# Пример получения значения NaN
ser = df[df['pay3'].isnull()].iloc[0]
print(ser.PAY_3, '/', ser.SHP_3, '= ', ser.pay3)

Если начислений нет, считаем, что все они оплачены.

In [ ]:
# Заполним пропуски 1, то есть коэффициентом полной оплаты
df['pay3'] = df['pay3'].fillna(1)
df['pay6'] = df['pay6'].fillna(1)
df['pay9'] = df['pay9'].fillna(1)
df['pay12'] = df['pay12'].fillna(1)
df['pay15'] = df['pay15'].fillna(1)
df['pay18'] = df['pay18'].fillna(1)

In [ ]:
# Проверим, сколько данных бесконечным значением
print('pay3 is infimum: ' , len(df[~np.isfinite(df['pay3'])]))
print('pay6 is infimum: ' , len(df[~np.isfinite(df['pay6'])]))
print('pay9 is infimum: ' , len(df[~np.isfinite(df['pay9'])]))
print('pay12 is infimum: ' , len(df[~np.isfinite(df['pay12'])]))
print('pay15 is infimum: ' , len(df[~np.isfinite(df['pay15'])]))
print('pay18 is infimum: ' , len(df[~np.isfinite(df['pay18'])]))

Бесконечные значения в сгенерированных столбцах могли появиться из-за того, что было деление ненулевого значения на ноль, то есть, сумма начислений была нулевой, а платежи > 0. То есть, платежи покрывали долги совсем старых периодов. Проверим это.

In [ ]:
print(df[~np.isfinite(df['pay3'])].SHP_3.unique())
print(df[~np.isfinite(df['pay6'])].SHP_6.unique())
print(df[~np.isfinite(df['pay9'])].SHP_9.unique())
print(df[~np.isfinite(df['pay12'])].SHP_12.unique())
print(df[~np.isfinite(df['pay15'])].SHP_15.unique())
print(df[~np.isfinite(df['pay18'])].SHP_18.unique())

# Пример получения значения Inf
ser = df[~np.isfinite(df['pay3'])].iloc[0]
ser.PAY_3
print(ser.PAY_3, '/', ser.SHP_3, '= ', ser.pay3)

In [ ]:
# Бесконечные значения заменим единицей, то есть коэффициентом полной оплаты
df['pay3'] = df['pay3'].apply(lambda x: 1 if not np.isfinite(x) else x)
df['pay6'] = df['pay6'].apply(lambda x: 1 if not np.isfinite(x) else x)
df['pay9'] = df['pay9'].apply(lambda x: 1 if not np.isfinite(x) else x)
df['pay12'] = df['pay12'].apply(lambda x: 1 if not np.isfinite(x) else x)
df['pay15'] = df['pay15'].apply(lambda x: 1 if not np.isfinite(x) else x)
df['pay18'] = df['pay18'].apply(lambda x: 1 if not np.isfinite(x) else x)

In [ ]:
# Проверим, нет ли случайно отрицательных платежей
df.pay3.min(), df.pay6.min(), df.pay9.min(), df.pay12.min(), df.pay15.min(), df.pay18.min()

In [ ]:
# Проверим количество отрицательных платежей
len(df[df.pay3<0]), len(df[df.pay6<0])

In [ ]:
# Удалим явно ошибочные строки с отрицательными платежами
df = df[(df.pay3>=0) & (df.pay6>=0)]
df.shape

### Признак DEB_TIME

У данного признака были пропуски. Посмотрим, сколько их и какие это данные.

In [ ]:
df[df.DEB_TIME.isnull()]

После отсечения малых долгов в этом столбце все данные оказались заполнены. 

### Признаки BU_COUNT и COURT_COUNT

In [ ]:
# Посмотрим на распределение данных признаков
bin_columns = ['BU_COUNT', 'COURT_COUNT']
i = 0
fig, ax = plt.subplots(1, 2, figsize=(8, 3))
for col in bin_columns:
    ax[i].hist(df[col], rwidth=0.9, alpha=0.7, bins=15)
    ax[i].set_title(col)
    i += 1
plt.show()

Как видно из графика, признак BU_COUNT оказался неинформативным, так как все его значения = 0. Скорее всего, это ошибка в скрипте, который использовался для набора данных. Исключим этот признак из моделирования.

### Распределение числовых признаков

Как видно было из сводного описания датасета, многие числовые признаки имели большой разброс в данных.

In [ ]:
# Проверим на распределение числовых данных, которые будут использоваться для кластеризации
for col in ['COURT_COUNT', 'DEB_TIME', 'control_days', 'overdue', 'pay3',
       'pay6', 'pay9', 'pay12', 'pay15', 'pay18']:
    val_log_plot(df, col)

Распределение после логарифмирования улучшилось либо осталось неизменным для всех числовых признаков. Есть смысл заменить все числовые признаки их логарифмом.

In [ ]:
# Логарифмируем числовые признаки
for col in ['COURT_COUNT', 'DEB_TIME', 'control_days', 'overdue', 'pay3',
       'pay6', 'pay9', 'pay12', 'pay15', 'pay18']:
        df[col+'_log'] = np.log(df[col]+1)

### Бинарные признаки

In [ ]:
# Посмотрим на распределение бинарных признаков
bin_columns = ['METER_OK', 'IS_IKUS', 'PHONE_OK', 'MOBILE_OK', 'EMAIL_OK']
i = 0
fig, ax = plt.subplots(1, 5, figsize=(18, 2))
for col in bin_columns:
    ax[i].hist(df[col], rwidth=0.9, alpha=0.7, bins=15)
    ax[i].set_title(col)
    i += 1
plt.show()
    

По графику видно, что признак METER_OK неинформативен, так как все его значения = 1. Исключим его из модели

#### Данные для кластеризации

In [ ]:
# Проверим столбцы, которые в итоге есть у датасета
df.columns

In [ ]:
# Выделим столбцы для применения кластеризации
df = df[['IS_IKUS', 'PHONE_OK', 'MOBILE_OK', 'EMAIL_OK', 
       'COURT_COUNT_log', 'DEB_TIME_log', 'control_days_log', 'overdue_log', 
       'pay3_log', 'pay6_log', 'pay9_log', 'pay12_log', 'pay15_log', 'pay18_log']] 

In [ ]:
# Проверим корреляцию столбцов в сборном датасете
correlation = df.corr()
plt.figure(figsize=(16, 12))
sns.heatmap(correlation, annot=True, cmap='coolwarm')

Тепловая карта корреляции показала, что наличие действующего адреса электронной почты и регистрации в личном кабинете сильно коррелируют друг с другом, что вполне объяснимо: именно из личного кабинета в подавляющем большинстве случаев приходит информация об электронном адресе. Столбец IS_IKUS исключим из модели.

Опытным путем было выяснено, что признак control_days_log (количество дней от последнего контрлольного посещения) на разбиение на кластеры совсем не влияет - во всех кластерах его распределение абсолютно одинаковое.Кроме того, при наличии этого признака области кластеров начинали пересекаться. В результате признак был удален из моделирования.      
Также опытным путем было выяснено, что наличие домашнего (стационарного) телефона тоже не оказывает влияния на кластеры. Домашние телефоны есть практически у всех, их номера получены компанией не за счет передачи их лично абонентом, а по телефонным справочникам советских годов. Признак был удален из моделирования.

In [ ]:
df = df[[ 'MOBILE_OK', 'EMAIL_OK', # 'IS_IKUS', 'control_days_log', 'PHONE_OK',
       'COURT_COUNT_log', 'DEB_TIME_log',  'overdue_log', 
       'pay3_log', 'pay6_log', 'pay9_log', 'pay12_log', 'pay15_log', 'pay18_log']] 

"Теплый" угол со столбцами уровня оплаты очень сильно коррелируют, особенно ближайшие соседи по периодам, но удалять их не будем, так как они демонстрируют динамику оплаты.

In [ ]:
# Нормализуем данные
scaler = StandardScaler()
df_sc = scaler.fit_transform(df)
df_sc

Датасет для кластеризации готов.

In [ ]:
# Запомним количество признаков
input_num = df_sc.shape[1] 

# Кластеризация

### Чистый ML: K-Means

Сначала применим классический алгоритм Machine Learning для кластеризации - это K-Means.      
Выбор этого метода кластеризации обусловлен:   
1) малым количеством кластеров,     
2) средним количеством признаков,    
2) необособленностью кластеров и их предполагаемой выпуклостью (скорее всего придется просто делить равномерное распределение объектов в пространстве признаков).

In [ ]:
# Кластеризуем
kmeans = KMeans(n_clusters=3,max_iter=300,random_state=RANDOM_SEED)
kmeans.fit(df_sc)   
labels1 = kmeans.labels_.astype(np.int)

In [ ]:
# Выведем на график кластеры и метрики кластеризации, предварительно понизив размерность методом выделения главных компонент 
centroids1 = plot_clusters(df_sc, labels1, name_alorithm = 'KMeans')
print('centroids: ')
print(centroids1)

Кластеры тесно смыкаются друг с другом, что означает, что пространство признаков должников довольно размыто и однородно.    


Метрики показывают не слишком хороший результат, но это обусловлено нечеткостью и размытостью состава должников.

In [ ]:
# Проверим распределение величин по каждому признаку для разных классов
df['cl_kmeans'] = labels1

for c in df:
    grid= sns.FacetGrid(df, col='cl_kmeans')
    grid.map(plt.hist, c)

In [ ]:
# Для быстрой интерпретации результата выведем сравнение средних значений по каждому кластеру/величин
for col in df.columns[:input_num]:
    fig, ax = plt.subplots(1, 2, figsize=(7,3))
    ax[0].set_title('Mean ' + col)
    df.groupby(by = 'cl_kmeans').mean()[col].plot(ax=ax[0], kind='bar')
    ax[1].set_title('Median ' + col)
    df.groupby(by = 'cl_kmeans').median()[col].plot(ax=ax[1], kind='bar')

In [ ]:
# Проверим количественное распределение
df['cl_kmeans'].value_counts()

Судя по распределению, кластеры были выделены так:    
**Кластер 0**:  небольшой долг, платят не слишком активно, технически продвинутые (есть e-mail и/или регистрация в личном кабинете, используют мобильное приложение и/или при оплате указывают мобильный телефон), судебных дел практически нет. В кластер 1 попало подавляющее большинство должников.       
**Кластер 1**: долг среднего размера или небольшой, платят очень активно, технически продвинуты, есть некоторое количество судебных дел.    
**Кластер 2**: большой долг, платят плохо, технически не продвинутые, много судебных дел. Это самый немногочисленный кластер, в него попали самые злостные маргинализированные абоненты.

Рекомендации для кластеров должников таковы:     
1) Для кластера 0 желательно оповестить о задолженности по всем доступным каналам (СМС, e-mail, личный кабинет). Так как абоненты из этого кластера пользуются гаджетами, уведомления с большой вероятностью будут ими прочитаны, а небольшой долг оплатить менее сложно, чем большой. Судебная работа не нужна.    
2) Для кластера 1 достаточно оповестить о задолженности по какому-либо одному каналу, а при недостатке средств возможно даже не оповещать, так как данная группа активно платит и тем самым сокращает долг. Уведомления будут прочитаны, так как кластер технически продвинут.    
3) Для кластера 2 лучше всего не тратить средства на дорогие оповещения, так как маловероятно, что они будут прочитаны, а лучше сразу выходить в суд, если до сих пор в суд не выходили. Если судебные дела уже ведутся, оповещать не нужно.

### Сеть Кохонена

Идея кластеризации с использованием нейронной сети Кохонена (она же сеть SOM - Self Organization Maps) заключается в том, чтобы преобразовать многомерное пространство признаков в более простое низкоразмерное пространство, сохранив при этом внутреннюю топологию данных. Фактические расстояния будут потеряны после преобразования SOM, но внутренняя структура данных сохранится.     
Сети Кохонена полезны, когда данных и их признаков очень много. За счет понижения размерности значительно сокращаются вычислительные ресурсы.

![Сеть Кохонена](https://ranalytics.github.io/data-mining/figures/cohonen_activation.png) 

Алгоритм кластеризации с помощью сети Кохонена таков:    
1) Подготовленный набор данных сначала прогоним через сеть SOM, получив на выходе карту с небольшими количеством ячеек.   
2) Карту SOM отправим на кластеризацию методом K-Means.

In [ ]:
# Создадим сеть Кохонена размером 20 на 20 выходных нейронов и активируем периодические граничные условия (PBC)
somModel = sps.somNet(20, 20, df_sc) #, PBC=True

# Обучим сеть в течение 1000 эпох с шагом learning rate = 0.01
somModel.train(0.01, 1000) 

In [ ]:
# Получим карту признаков сниженной размерности, спроектировав наш датасет на плоскость при помощи обученной сети 
map_ = np.array((somModel.project(df_sc)))

In [ ]:
# Проверим размерность карты
map_.shape

Визуализируем полученную карту. На графиках весов и расстояний между ячейками карты уже будут заметны области с разными характеристиками. Их алгоритм K-Means, скорее всего, и выделит в отдельные кластеры. Проверим это.

In [ ]:
# Визуализируем веса каждой ячейки карты
somModel.nodes_graph(colnum=0)

In [ ]:
# Визуализируем расстояния каждой ячейки карты до ее соседей
somModel.diff_graph(show=True,printout=True)

In [ ]:
# Кластеризуем преобразованные данные
kmeans = KMeans(n_clusters=3,max_iter=300,random_state=RANDOM_SEED)
kmeans.fit(map_)   
labels2 = kmeans.labels_.astype(np.int)

In [ ]:
# Визуализируем кластеры
centroids2 = plot_clusters(map_, labels2, need_pca=False)
print('centroids:')
print(centroids2)

В принципе, данный график повторяет области на картах с весами и расстояниями.

In [ ]:
# Выведем информацию из разных кластеров, чтобы можно было интерпретировать результат
df['cl_som'] = labels2

for col in df.columns[:input_num]:
    fig, ax = plt.subplots(1, 2, figsize=(7,3))
    ax[0].set_title('Mean ' + col)
    df.groupby(by = 'cl_som').mean()[col].plot(ax=ax[0], kind='bar')
    ax[1].set_title('Median ' + col)
    df.groupby(by = 'cl_som').median()[col].plot(ax=ax[1], kind='bar')

In [ ]:
# Проверим количественное распределение кластеров
df.cl_som.value_counts()

По данным графикам видно, что сеть Кохонена разделила должников по иному принципу - по принципу их оплаты. Кластеры четко соответствуют динамике оплаты - от самой низкой оплаты до самой высокой. К сожалению, в API библиотеки simpSOM нет возможности зафиксировать начальную инициализацию весов (она всегда происходит случайно) и тем самым установить воспроизводимость результата. Поэтому разбиение на кластеры может получаться иным с каждым новым прогоном через сеть. Но в нашем случае сеть Кохонена стабильно делит на кластеры  по принципу их оплаты. При этом разбиение по сумме долга и наличию судебных дел может меняться.    
    
Чаще всего получается кластеризация, совпадающая с результатами K-Means:      
**Кластер 0:** должники с самым большим долгом и самыми маленькими оплатами, технически не продвинутые, судебных дел много. То есть, это злостные маргинализированные неплательщики.         
**Кластер 1:** должники с самым маленьким уровнем долга, с не слишком активными оплатами. Технически продвинуты, судебных дел мало.     
**Кластер 2:** должники со средним уровнем долга, с самыми активными оплатами, судебных дел среднее количество, технически  продвинутые.  
Рекомендации по ведению работы с ними тоже совпадают с рекомендациями для разбиения методом K-Means:     
Для кластера 0 - оповещение не имеет смысла, необходимо судиться, если до сих пор не вышли в суд, и ничего не делать, если в суд уже вышли.
Для кластера 1 - оповещать по всем каналам. В суд идти не надо.
Для кластера 2 - оповестить по одному каналу, а при недостатке средств вовсе не оповещать, так как данный кластер неплохо оплачивает долги. В суд идти не надо, достаточно досудить старые дела.
     
Но также встречается второй вариант кластеризации:    
**Кластер 0:** должники с самым большим долгом и самыми большими оплатами, технически не продвинутые, судебных дел много. То есть, это должники, с которых принудительно через приставов уже взыскиваются долги. Рекомендация - ничего не делать с ними, поскольку суды уже идут и долги постепеннно погашаются.               
**Кластер 1:** должники со средним уровнем долга и с малым уровнем оплат. Технически продвинуты средне, судебных дел мало. Рекомендации - оповещать по всем каналам, если есть средства оповещения, и выходить в суд, если средств оповещения нет.          
**Кластер 2:** должники с малым долгом и средним уровнем оплаты, судебных дел практически нет, технически  продвинутые. Рекомендации - оповестить по одному каналу.

Но в целом разбиение каждый раз новое и интерпретируется тоже каждый раз по-новому.

In [ ]:
# Проверим распределение величин по каждому признаку для разных классов
for c in df:
    grid= sns.FacetGrid(df, col='cl_som')
    grid.map(plt.hist, c)

### Autoencoder

Идея кластеризации с использованием нейронной сети типа Autoencoder тоже, как и в случае сети Кохонена, заключается в том, чтобы преобразовать многомерное пространство признаков в более простое низкоразмерное пространство, сохранив при этом глубинные характеристики данных. Для этого строится нейронная сеть с двумя частями: декодер, который сворачивает размерность до заданного уровня и энкодер, который восстанавливает размерность. "Бутылочное горлышко", которое получается на слое с минимальной размерностью (скрытый слой, latent dense), подается в классический алгоритм кластеризации.        
Сеть Autoencoder полезна, когда признаков очень много. За счет понижения размерности значительно сокращаются вычислительные ресурсы. Кроме того, уровень скрытого слоя хранит самую существенную информацию об объекте и кластеризация на его основе проводится более чисто.

![](https://www.researchgate.net/profile/Kamran-Kowsari/publication/332330221/figure/fig2/AS:746162701217795@1554910459741/Structure-of-clustering-model-with-autoencoder-and-K-means-combination.png)

In [ ]:
df_sc.shape

In [ ]:
# Зададим параметры для автоэнкодера
encoding_dim = 6           # количество нейронов в "бутылочном горлышке"
input_num = df_sc.shape[1] # количество признаков

In [ ]:
# Построим сеть для автоэнкодера

# Кодирование
input_df = Input(shape=(input_num,)) 
x = Dense(encoding_dim, activation='relu')(input_df)
x = Dense(500, activation='relu')(x)
x = Dense(1000, activation='relu')(x)
x = Dense(2000, activation='relu')(x)
encoded = Dense(encoding_dim, activation='relu')(x)
encoder = Model(input_df, encoded)

# Декодирование
x = Dense(2000, activation='relu')(encoded)
x = Dense(1000, activation='relu')(encoded)
x = Dense(500, activation='relu')(x) # попробовать другую функцию активации
decoded = Dense(input_num)(x)
autoencoder = Model(input_df, decoded)

# Компилируем автоэнкодер
autoencoder.compile(optimizer= 'adam', loss='mean_squared_error')

In [ ]:
# Описываем callback
# Так как нам для кластеризации, в отличие от предсказания не требуется идеальная точность,
#    управлять шагом обучения (снижать на плато/задавать шедулер) не станем. 
checkpoint = ModelCheckpoint('../working/best_model.hdf5' , monitor=['loss'], verbose=0  , mode='min')
earlystop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True,)
# callbacks_list = [checkpoint, earlystop]
callbacks_list = [earlystop]

In [ ]:
# Обучаем сеть
autoencoder.fit(df_sc, df_sc, 
                batch_size = 128, 
                epochs = 100,  
                callbacks=callbacks_list,
                verbose = 1)

In [ ]:
# Кодируем наш набор данных для снижения размерности и получения скрытого слоя
pred = encoder.predict(df_sc)

In [ ]:
# Функция для сбора методов кластеризации
# Кластеризуем только теми методами, у которых можно задать количество кластеров
def generate_clustering_algorithms(Z,n_clusters):
    kmeans = cluster.KMeans(n_clusters=n_clusters, random_state = RANDOM_SEED)
    agglomerative = cluster.AgglomerativeClustering(n_clusters=n_clusters)
    birch = cluster.Birch(n_clusters=n_clusters)
    gmm = mixture.GaussianMixture(n_components=n_clusters)

    clustering_algorithms = (
        ('KMeans', kmeans),
        ('AgglomerativeClustering', agglomerative),
        ('GaussianMixture', gmm),
        ('Birch', birch)
    )
    return clustering_algorithms

# Кластеризуем данные скрытого слоя и визуализируем разбиение
centroids = []
clustering_algorithms = generate_clustering_algorithms(pred,3)

plt.figure(figsize=(10 * 2 + 2, 15))
plt.subplots_adjust(left=.02, right=.98, bottom=.01, top=.98, wspace=.05,
                    hspace=.01)

for name, algorithm in clustering_algorithms:
    algorithm.fit(pred)

    if hasattr(algorithm, 'labels_'):
        lbls = algorithm.labels_.astype(np.int)
    else:
        lbls = algorithm.predict(pred)       
        
    df['cl_ae_'+name[:2]] = lbls

    centroid = plot_clusters(pred, lbls, name_alorithm = name)
    centroids.append(centroid)

Все разбиения схожи, кроме разбиения методом BIRCH.

In [ ]:
# Пространство объектов имеет структуру с неявно выраженными кластерами. 
# Проверим, как разобьют на кластеры методы, в которых их количество определяется автоматически
dbs = DBSCAN(eps=0.7, min_samples=90)
lbls = dbs.fit_predict(pred)
cDBSC = plot_clusters(pred, lbls)

op = cluster.OPTICS(eps=0.8, min_samples=10)
op.fit_predict(pred)
lbls = op.labels_
cOPT = plot_clusters(pred, lbls)

Оба этих метода дают крайне неподходящий вариант разбиения, что свидетельствует о рыхлой структуре множества без явно выделенных сегментов.

In [ ]:
# Находим попарные расстояния между центроидом разбиения 0 и центроидами остальных разбиений (кроме BIRCH)
d_mat1 = spatial.distance.cdist(np.array(centroids[0]), np.array(centroids[1]))
d_mat2 = spatial.distance.cdist(np.array(centroids[0]), np.array(centroids[2]))

# Выбираем наиболее маленькие расстояния в каждой строке
# Индекс самого маленького расстояния - это номер кластера в разбиении i, 
#    который соответствует кластеру в разбиении 0.
# Иными словами, мы красим (именуем) кластеры в разных разбиениях так же, как они покрашены (поименованы) в разбиении 0
dict1 = {}
for i in range(d_mat1.shape[0]):
    dict1[i] = np.argmin(d_mat1[i])
dict2 = {}
for i in range(d_mat2.shape[0]):
    dict2[i] = np.argmin(d_mat2[i])

# Перекрашиваем кластеры
df['cl_ae_Ag'] = df['cl_ae_Ag'].map(dict1)
df['cl_ae_Ga'] = df['cl_ae_Ga'].map(dict2)

Теоретически может оказаться, что 2 минимальных значения из разных строк оказались в одном столбце, но тогда можно в качестве образца по очереди взять другие деления на кластеры и сравнивать с ними, добиваясь, чтобы столбцы не повторялись.

In [ ]:
# Проверим количественные распределения кластеров в разных разбиениях
display(df['cl_ae_KM'].value_counts())
display(df['cl_ae_Ga'].value_counts())
display(df['cl_ae_Ag'].value_counts())
display(df['cl_ae_Bi'].value_counts())

Проверим распределения отдельных признаков для разного вида кластеров в каждом разбиении

In [ ]:
# K-Means
for col in df.columns[:7]:
    fig, ax = plt.subplots(1, 2, figsize=(7,3))
    ax[0].set_title('Mean ' + col)
    df.groupby(by = 'cl_ae_KM').mean()[col].plot(ax=ax[0], kind='bar')
    ax[1].set_title('Median ' + col) 
    df.groupby(by = 'cl_ae_KM').median()[col].plot(ax=ax[1], kind='bar')

In [ ]:
# Aggl
for col in df.columns[:7]:
    fig, ax = plt.subplots(1, 2, figsize=(7,3))
    ax[0].set_title('Mean ' + col)
    df.groupby(by = 'cl_ae_Ag').mean()[col].plot(ax=ax[0], kind='bar')
    ax[1].set_title('Median ' + col)
    df.groupby(by = 'cl_ae_Ag').median()[col].plot(ax=ax[1], kind='bar')

In [ ]:
# GM
for col in df.columns[:7]:
    fig, ax = plt.subplots(1, 2, figsize=(7,3))
    ax[0].set_title('Mean ' + col)
    df.groupby(by = 'cl_ae_Ga').mean()[col].plot(ax=ax[0], kind='bar')
    ax[1].set_title('Median ' + col)
    df.groupby(by = 'cl_ae_Ga').median()[col].plot(ax=ax[1], kind='bar')

In [ ]:
# Делаем ансамбль по голосованию
clusters = []
clusters.append(df['cl_ae_KM'].values)
clusters.append(df['cl_ae_Ga'].values)
clusters.append(df['cl_ae_Ag'].values)

clusters = scipy.stats.mode(clusters)[0]
clusters[0]
df['cl_ae_mode'] = clusters[0]
df['cl_ae_mode'].value_counts()

In [ ]:
# Распределение в ансамбле
for col in df.columns[:7]:
    fig, ax = plt.subplots(1, 2, figsize=(7,3))
    ax[0].set_title('Mean ' + col)
    df.groupby(by = 'cl_ae_mode').mean()[col].plot(ax=ax[0], kind='bar')
    ax[1].set_title('Median ' + col)
    df.groupby(by = 'cl_ae_mode').median()[col].plot(ax=ax[1], kind='bar')

In [ ]:
# визуализируем ансамбль
centroids_ens = plot_clusters(pred, df['cl_ae_mode'].values)

# Выдача результатов по сегментации должников

Результатом анализа должны стать списки абонентов с рекомендацией для каждого по необходимости оповещения о задолженности и/или необходимости подачи на них в суд. Для этого для каждого кластера будут рассчитаны простейшие статистики и на основании этих статистик (средних величин) будут выданы рекомендации. Рекомендации будут даваться в соответствии с приложенным шаблоном действий. Шаблон заполнялся заказчиком и описывает, что надо делать с должником в зависимости от средних статистик его кластера.

В результате будут выданы 4 возможные рекомендации:     
1) Для кластеров, полученных простым применением KMeans (колонка с этим разбиением = cl_kmeans),      
2) Для кластеров, полученных понижением размерности при помощи SOM-сети Кохонена (колонка с этим разбиением = cl_som),     
3) Для кластеров, полученных понижением размерности при помощи автоэнкодера и ансамбля кластерных методов (колонка с этим разбиением = cl_ae_mode),     
4) Для кластеров, полученных понижением размерности при помощи автоэнкодера и кластерного метода BIRCH (колонка с этим разбиением = cl_ae_Bi).

Из этих четырех разбиений должников на сегменты пользователь самостоятельно выберет тот, что кажется ему более подходящим.

### Подсчет статистики по признакам и применение шаблона действий

In [ ]:
# Вычисляем статистики, то есть условия применения действия в шаблоне для разных разбиений
df_res = df.copy()
for col in ['cl_kmeans', 'cl_som', 'cl_ae_mode', 'cl_ae_Bi']:
    # Оснащенность кластера мобильными телефонами и электронной почтой
    a = df_res.groupby(by = col).EMAIL_OK.mean()
    b = df_res.groupby(by = col).MOBILE_OK.mean()
    geek = np.argmin(a+b)
    df_res['geek'] = df_res[col].apply(lambda x: 0 if x==geek else 1)
    
    # Наличие судебных дел
    pir = np.argmax(df_res.groupby(by = col).COURT_COUNT_log.mean())
    df_res['pir'] = df_res[col].apply(lambda x: 1 if x==pir else 0)
    
    # Наличие большого долга
    big_overdue = np.argmax(df_res.groupby(by = col).overdue_log.mean())
    df_res['big_overdue'] = df_res[col].apply(lambda x: 1 if x==big_overdue else 0)
    
    # Отсутствие оплат
    pay3 = np.argmin(df_res.groupby(by = col).pay3_log.mean())
    pay6 = np.argmin(df_res.groupby(by = col).pay6_log.mean())
    pay9 = np.argmin(df_res.groupby(by = col).pay9_log.mean())
    pay12 = np.argmin(df_res.groupby(by = col).pay12_log.mean())
    pay15 = np.argmin(df_res.groupby(by = col).pay15_log.mean())
    pay18 = np.argmin(df_res.groupby(by = col).pay18_log.mean())

    payments = scipy.stats.mode([pay3,pay6,pay9,pay12,pay15,pay18])[0]
    p = payments[0]
    df_res['not_little_pay'] = df_res[col].apply(lambda x: 0 if x==p else 1)
    
    # Применение шаблона
    df_res = df_res.merge(pattern, on=['pir', 'big_overdue', 'not_little_pay', 'geek'], how='inner')
    
    # Переименование столбцов для следующей итерации цикла
    df_res = df_res.rename(columns = {'geek': 'geek_'+col, 
                                      'pir': 'pir_'+col,
                                      'big_overdue': 'big_overdue_'+col,
                                      'not_little_pay': 'not_little_pay_'+col,
                                      'need_message': 'need_message_'+col,
                                      'need_pir': 'need_pir_'+col
                                     })

In [ ]:
# Применяем шаблон по вычисленной статистике и выводим на графике распределение
for col in ['cl_kmeans', 'cl_som', 'cl_ae_mode', 'cl_ae_Bi']:
    tt = pd.DataFrame(columns=['Метод','Количество'])
    for nm in pattern.need_message.unique():
        for np in pattern.need_pir.unique():
            temp = df_res[(df_res['need_message_'+col]==nm) & (df_res['need_pir_'+col]==np)]
            tt.loc[len(tt),'Метод'] = 'Оповещение = ' + nm + '. Судебная работа = ' + np
            tt.loc[len(tt)-1,'Количество'] = len(temp)
    ax = tt.groupby(by = ['Метод']).sum().plot.barh()
    ax.set_title('Результаты для ' + col)

### Сохранение результата

In [ ]:
df_res.to_csv('DebtorSegmentation.csv', index=False)

# Заключение

**Решение поставленной задачи было разбито на следующие этапы:**     
1. Создание SQL-запроса к рабочей базе данных и выгрузка в файл информации по должникам;     
2. Чтение, анализ и преобразование данных из файла информации по должникам;    
3. Подготовка данных для кластеризации;       
4. Кластеризация классическим ML-методом K-Means;     
5. Понижение размерности пространства признаков с применением DL-методов;     
6. Кластеризация пространства пониженной размерности набором ML-методов и ансамблирование этих методов;     
7. Определение метрик кластеризации: обособленность (силуэт), однородность, полнота и v-мера (однородность + полнота);    
7. Подсчет статистик по кластерам и применение шаблона работы с должниками на основании этих статистик.  

**В ходе подготовки данных для кластеризации было выполнено:**     
* Преобразование строковых данных в числовые форматы;     
* Визуализация данных и проверка распределения;    
* Логарифмирование числовых данных для приближения распределения к нормальному;    
* Частичное удаление выбросов по критически важному признаку;    
* Генерация новых признаков;     
* Проверка корреляции и удаление из модели корелирующих признаков;    
* Нормализация данных при помощи StandardScaler.

**Кластеризация методом K-Means показала:**     
* Пространство признаков достаточно рыхлое и распределено без явных обособленных кластеров. Метрики это подтверждают.      
* Метрики кластеризации:      
  - silhouette =  0.28     
  - homogeneity =  0.18      
  - completeness = 1.00    
* Превалирующими факторами в сегментировании стали величина долга и стремление его оплатить. Явно была выделена группа абонентов, имеющих большой долг и не желающих его оплачивать.    
* Факторы наличия контактных данных на сегментирование повлияло мало. 

**Шаги кластеризации с понижением размерности при помощи SOM-сети:**   
* Применение сети Кохонена с использованием библиотеки simpSOM;     
* Визуализация построенной сети: весов полученных ячеек сети и расстояний между ячейками;     
* Кластеризация полученных ячеек методом R-Means;    
* Визуализация результатов кластеризации;    
* Вычисление метрик близости и однородности кластеров:
  - silhouette = 0.46
  - homogeneity = 0.23
  - completeness = 0.55   
  Метрики могут незначительно меняться, так как зафиксировать воспроизводимость результатов при применении simpSOM невозможно.    
* Расчет и визуализация распределения значений признаков для разных кластеров.

**Шаги кластеризации с размерности при помощи автокодировщика:**    
* Построение нейронной сети типа Autoencoder. Для кодировщика были взяты 3 плотных слоя с нарастанием количества нейронов + добавлен скрытый слой с малым количеством нейронов ("бутылочное горлышко"). Затем для декодирования были взяты 3 плотных слоя с уменьшением количества нейронов.    
* Обучение автокодировщика с применением callback для ранней остановки при выходе на плато метрики loss.     
* Получение предсказания кодировщиком на пространстве признаков с выходом в виде скрытого слоя ("бутылочное горлышко" пониженной размерности).    
* Кластеризация предсказания пониженной размерности методами ML с указанием количества кластеров:    
  - K-Means (silhouette=0.48, homogeneity=0.10, completeness=1.00),    
  - Agglomerative Clustering (silhouette=0.47, homogeneity=0.10, completeness=1.00),     
  - Birch (silhouette=0.45, homogeneity=0.05, completeness=1.00),   
  - Gaussian Mixture (silhouette=0.41, homogeneity=0.11, completeness=1.00).  
* Разведывательная кластеризация методами, автоматически определяющими количество кластеров:    
  - DBSCAN,   
  - OPTICS.    
  Данные методы показали крайне неудовлетворительный результат по сегментации и метрикам, что свидетельствует о плохо структурированном пространстве признаков.     
* Ансамблирование по голосованию для 3 методов, показавших схожие результаты (K-Means, Agglomerative Clustering, Gaussian Mixture) с предварительным переименованием кластеров. Отнесение кластера к единому шаблону выполнялось на основании близости центроидов. Метрики ансамбля:       
  - silhouette = 0.48
  - homogeneity = 0.10
  - completeness = 1.00    
  Кластеризация методом BIRCH выдало разбиение, существенно отличающееся от трех других. В связи с этим данная сегментация рассматривалась отдельно и не включалась в ансамбль.

**Выдача результатов сегментирования заключалась в следующем:**   
* Для каждого кластера из каждого способа разбиения (K-Means, SOM, ансамбль по автокодировщику, BIRCH по автокодировщику) были рассчитаны статистики и в зависимости от них указаны признаки:   
  - Кластер с самым большим количеством судебных дел,     
  - Кластер с самой большой суммой долга,    
  - Кластер с самой маленькой суммой оплаты,   
  - Кластер с самым большим количеством контактной информации.    
* Указания, какие методы работы с должником применять в зависимости от его признаков, были даны в файле с шаблоном определения методов. Данный шаблон разрабатывался отделом работы с должниками. Шаблон был наложен на рассчитанные признаки по каждому должнику и по нему определена рекомендация.      
* Рекомендации с обоснованием их выдачи были сохранены в итоговый файл.     
* Рекомендация по каждому должнику была дана для каждого спсоба сегментрирования, то есть в 4-х вариантах. Выбор варианта оставлен за пользователем, то есть, за сотрудником отдела работы с должниками.   